# Transformer (tylko decoder), inny encoding. 5 featerow - mag, x, y, depth, diff_days

In [20]:
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [21]:
ds_train = tf.data.Dataset.load("../data/ds_final_train/")
ds_val = tf.data.Dataset.load("../data/ds_final_val/")
ds_test = tf.data.Dataset.load("../data/ds_final_test/")

In [22]:
BATCH_SIZE = 1024
BLOCK_SIZE = 64

In [23]:
def transform_element(features, context, labels):
    # transform lebels from (1,) to (2,)
    labels = tf.cast(labels, tf.int32)
    labels = tf.one_hot(labels, depth=2)
    return (features[:,:-2], features[:,-2], features[:,-1]), labels[0]

In [24]:
ds_train = ds_train.map(transform_element)
ds_val = ds_val.map(transform_element)
ds_test = ds_test.map(transform_element)

In [25]:
for x, y in ds_train.take(1):
    print(x.shape)
    print(y.shape)

AttributeError: 'tuple' object has no attribute 'shape'

In [26]:
ds_train = ds_train.batch(BATCH_SIZE).shuffle(2000, seed=SEED, reshuffle_each_iteration=False).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [31]:
class MyModel(tf.keras.Model):
    def __init__(self, n_embed):
        super(MyModel, self).__init__()
        self.lstm_embed = tf.keras.layers.LSTM(n_embed // 7 * 5, return_sequences=True)
        self.embed_plate = tf.keras.layers.Embedding(100, n_embed // 7)
        self.embed_dd = tf.keras.layers.Embedding(100, n_embed // 7)
        self.conc = tf.keras.layers.Concatenate()
        self.lstm1 = tf.keras.layers.LSTM(n_embed, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(n_embed)
        self.d1 = tf.keras.layers.Dense(n_embed)
        self.d2 = tf.keras.layers.Dense(2)

    def call(self, inputs):
        cont, plate, dd = inputs
        x1 = self.lstm_embed(cont)
        x2 = self.embed_plate(plate)
        x3 = self.embed_dd(dd)
        x = self.conc([x1, x2, x3])
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.d1(x)
        return self.d2(x)
    

In [32]:
model = MyModel(7 * 9)
model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [33]:
history = model.fit(ds_train, epochs=20, validation_data=ds_val)

Epoch 1/20


2023-11-14 11:01:41.905763: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 83 of 2000
2023-11-14 11:01:51.950087: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 154 of 2000
2023-11-14 11:02:11.904293: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 297 of 2000
2023-11-14 11:02:31.894976: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 445 of 2000
2023-11-14 11:02:41.938764: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 523 of 2000
2023-11-14 11:03:01.908015: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a w

3355/3355 [==============================] - 879s 161ms/step - loss: 0.2633 - accuracy: 0.9058 - f1_score: 0.7625 - val_loss: 0.2466 - val_accuracy: 0.9133 - val_f1_score: 0.7255
Epoch 2/20


2023-11-14 11:16:07.193444: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 76 of 2000
2023-11-14 11:16:17.268808: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 145 of 2000
2023-11-14 11:16:27.454713: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 224 of 2000
2023-11-14 11:16:47.210496: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 383 of 2000
2023-11-14 11:16:57.255017: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 462 of 2000
2023-11-14 11:17:17.261580: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a w

3355/3355 [==============================] - 823s 172ms/step - loss: 0.2508 - accuracy: 0.9091 - f1_score: 0.7709 - val_loss: 0.2494 - val_accuracy: 0.9135 - val_f1_score: 0.7218
Epoch 3/20


2023-11-14 11:29:50.571977: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 54 of 2000
2023-11-14 11:30:10.549804: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 186 of 2000
2023-11-14 11:30:20.594981: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 246 of 2000
2023-11-14 11:30:30.596596: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 313 of 2000
2023-11-14 11:30:50.587705: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 454 of 2000
2023-11-14 11:31:00.674258: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a w

3355/3355 [==============================] - 888s 176ms/step - loss: 0.2466 - accuracy: 0.9096 - f1_score: 0.7701 - val_loss: 0.2507 - val_accuracy: 0.9138 - val_f1_score: 0.7200
Epoch 4/20


2023-11-14 11:44:38.510869: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 64 of 2000
2023-11-14 11:44:58.507601: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 208 of 2000
2023-11-14 11:45:18.431905: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 349 of 2000
2023-11-14 11:45:28.517735: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 415 of 2000
2023-11-14 11:45:38.554118: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 481 of 2000
2023-11-14 11:45:58.423566: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a w

  50/3355 [..............................] - ETA: 6:56 - loss: 0.2362 - accuracy: 0.9200 - f1_score: 0.6819

In [ ]:
model.evaluate(ds_test)

100/100 [==============================] - 14s 138ms/step - loss: 0.1349 - accuracy: 0.9553 - f1_score: 0.7734


[0.13489733636379242,
 0.9553054571151733,
 array([0.9764265, 0.5703992], dtype=float32)]

In [ ]:
model.save("../models/model_v7.keras")

In [ ]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/history_model_v7.csv", index=False)